---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

## Imports and Data

In [1]:
import numpy as np
import pandas as pd
import re
pd.set_option('display.max_colwidth', 100)

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

s = pd.Series(doc)
s.head()

0                        03/25/93 Total time of visit (in minutes):\n
1                                      6/18/85 Primary Care Doctor:\n
2            sshe plans to move as of 7/8/71 In-Home Services: None\n
3                               7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n
dtype: object

## Workspace

In [2]:
df = pd.DataFrame(data=s, columns=['string'])

df['original_index'] = df.index
df = df.reindex_axis(['original_index', 'string'], axis=1)  # Re-order columns

df.head()

original_index  \
0               0   
1               1   
2               2   
3               3   
4               4   

                                                             string  
0                      03/25/93 Total time of visit (in minutes):\n  
1                                    6/18/85 Primary Care Doctor:\n  
2          sshe plans to move as of 7/8/71 In-Home Services: None\n  
3                             7 on 9/27/75 Audit C Score Current:\n  
4  2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n

### Rows 0-124
mm/dd/yy OR mm/dd/yyyy

In [3]:
s[0:126].head()

0                        03/25/93 Total time of visit (in minutes):\n
1                                      6/18/85 Primary Care Doctor:\n
2            sshe plans to move as of 7/8/71 In-Home Services: None\n
3                               7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (Numeric Scale): 7\n
dtype: object

In [4]:
first = df['string'].str.extractall(r'.*(?P<mm>\d{1,2})[/-](?P<dd>\d{1,2})[/-](?P<yyyy>\d{2,4}).*')
first[0:125]

mm  dd  yyyy
    match             
0   0      3  25    93
1   0      6  18    85
2   0      7   8    71
3   0      9  27    75
4   0      2   6    96
5   0      7  06    79
6   0      5  18    78
7   0      0  24    89
8   0      3   7    86
9   0      4  10    71
10  0      5  11    85
11  0      4  09    75
12  0      8  01    98
13  0      1  26    72
14  0      5  24  1990
15  0      1  25  2011
16  0      4  12    82
17  0      0  13  1976
18  0      4  24    98
19  0      5  21    77
20  0      7  21    98
21  0      0  21    79
22  0      3  03    90
23  0      2  11    76
24  0      7  25  1984
25  0      4  13    82
26  0      9  22    89
27  0      9  02    76
28  0      9  12    71
29  0      0  24    86
...       ..  ..   ...
95  0      4  11  1974
96  0      7  18    86
97  0      3  31    91
98  0      5  13    72
99  0      1  14    83
100 0      8  16    92
101 0      0  05    97
102 0      7  18  2002
103 0      9  22    82
104 0      2  24    74
105 0      2  03    78
106 0      2  11  2006
107 0      8  22    83
108 0      5  04    74
109 0      7  20  2011
110 0      6  17    95
111 0      6  10    72
112 0      0  16    82
113 0      2  15    92
114 0      2   8    97
115 0      4  05    89
116 0      2  04    87
117 0      6  20    77
118 0      4  27  2006
119 0      7  17    92
120 0      2  22    98
121 0      0  02    96
122 0      1  05    90
123 0      5  04    77
124 0      2  27    96

[125 rows x 3 columns]

### Rows 125-193 AND 228-342
dd mon yyyy OR mon yyyy

In [426]:
s.loc[125:194].head()
s.loc[228:343].head()

125    s The patient is a 44 year old married Caucasian woman, unemployed Decorator, living with husban...
126                                           .10 Sep 2004 - Intake at EEC for IOP but did not follow up\n
127                          see above and APS eval of 26 May 1982 Social History Marital Status: Single\n
128                        Tbooked for intake appointment at Sierra Vista, Chongging, WY on 28 June 2002\n
129                                                                      06 May 1972 SOS-10 Total Score:\n
dtype: object

In [621]:
s.loc[[233, 240, 244, 270, 273, 306, 311, 317, 321, 328, 329, 338, 339]]

233                              Dr. Gloria English, who conducted an initial consultation in July, 1990\n
240            )- Venlafaxine 37.5mg daily: May, 2011: self-discontinued due to side effects (dizziness)\n
244    s Mr. Moss is a 27-year-old, Caucasian, engaged veteran of the Navy. He was previously scheduled...
270                                                                       May, 2006 Primary Care Doctor:\n
273            ) - Zoloft 100 mg daily: February, 2010 : self-discontinued due to side effects (unknown)\n
306                                                                     May, 2004 Hx of Brain Injury: No\n
311               - Prozac 20 mg daily:  February, 1995: self-discontinued due to side effects (unknown)\n
317    . Psychosocial: lives w/ father, looking to get own apartment. Lives in Saluda. Fa. has roommate...
321                                                                   2June, 1999 Audit C Score Current:\n
328    s Pt reports long Hx of drug a

In [680]:
s.loc[328]

's Pt reports long Hx of drug addiction. PLEASE SEE NSC SUPPLEMENTAL NOTE on this same date for details about substance use Hx. Her drugs of choice are opiates and benzodiazepines. She said that she used cocaine about once per month when she was on methadone. She got off of methadone in May, 2001 (7 months ago). She reports using it cocaine 1 x since getting off methadone. She states she has been abstinent from opiates for 7 months. Pt states: "I have no desire to use an opiate or a drug at all. I don\'t have a desire to use heroin or have any craving for it. I can\'t stand that I am all of the place, I can\'t have a conversation, I can\'t sit down and watch a movie - my ADHD is that bad. I just want my ADHD treated. The heroin takes the racing thoughts away."  She is primarily seeking treatment for ADHD. She reps her substance abuse has been to self-medicate her ADHD and believes that if she were treated for ADHD she would stop abusing substances. Currently, Pt\'s PCP, Dr. Michael Yar

In [699]:
second = df['string'].str.extractall('(?P<dd>\d{1,2} )?(?P<mm>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(?:[a-z]* )?(?P<yyyy>\d{4})')

second = df['string'].str.extractall('(?P<dd>\d{1,2} )?(?P<mm>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(?:[a-z]* )?(?:\D*)(?P<yyyy>\d{4})')

# second.loc[125:194]
second.loc[228:343]
second.loc[[233, 240, 244, 270, 273, 306, 311, 317, 321, 328, 329, 338, 339]]

dd   mm  yyyy
    match                
233 0      NaN  Jul  1990
240 0      NaN  May  2011
244 0      NaN  Jan  2013
270 0      NaN  May  2006
273 0      NaN  Feb  2010
306 0      NaN  May  2004
311 0      NaN  Feb  1995
317 0      NaN  Mar  1975
321 0      NaN  Jun  1999
328 0      NaN  May  2001
329 0      NaN  Mar  2000
338 0      NaN  Apr  1998
339 0      NaN  Mar  2005

### Rows 194-227
mon dd yyyy

In [428]:
s.loc[194:228].head()

194                                                  April 11, 1990 CPT Code: 90791: No medical services\n
195    MRI May 30, 2001 empty sella but no problems with endocrine functionPertinent Medical Review of ...
196    .Feb 18, 1994: made a phone call to Mom and Mom commented that he was talking very fast, hard to...
197                                       Brother died February 18, 1981 Parental/Caregiver obligations:\n
198    none; but currently has appt with new HJH PCP Rachel Salas, MD on October. 11, 2013 Other Agency...
dtype: object

In [602]:
third = df['string'].str.extractall('(?P<mm>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(?:.*) (?P<dd>\d{1,2}).? (?P<yyyy>\d{4})')
third.loc[194:228]

34

### Rows 343-454
mm/yyyy

In [432]:
s[343:455].head()

343                                                                          6/1998 Primary Care Doctor:\n
344    s 52 y/o MWM h/o chronic depression, anxiety, adhd.  Here for psychopharm transfer in the contex...
345                                                                      10/1973 Hx of Brain Injury: Yes\n
346                                                                          9/2005 Primary Care Doctor:\n
347                                                s 03/1980 Positive PPD: treated with INH for 6 months\n
dtype: object

In [490]:
fifth = df['string'].str.extractall('(?P<mm>\d{1,2})/(?P<yyyy>\d{4})')
fifth.loc[343:455]

mm  yyyy
    match          
343 0       6  1998
344 0       6  2005
345 0      10  1973
346 0       9  2005
347 0      03  1980
348 0      12  2005
349 0       5  1987
350 0       5  2004
351 0       8  1974
352 0       3  1986
353 0      10  1997
354 0       3  1993
355 0       3  1981
356 0       9  2003
357 0      10  1993
358 0       1  1983
359 0       7  1994
360 0      12  2008
361 0      10  1980
362 0       8  2003
363 0      12  1975
364 0      11  2010
365 0       7  1997
366 0       7  2014
367 0       9  2001
368 0       8  1986
369 0       1  1978
370 0       9  1975
371 0       1  2009
372 0       5  1995
...        ..   ...
425 0       7  2004
426 0      11  1984
427 0       5  2016
428 0      12  2007
429 0      11  1982
430 0      06  1981
431 0       4  2013
432 0       4  1999
433 0       5  2006
434 0      10  1978
435 0      12  1989
436 0       2  1974
437 0      11  1986
438 0       4  1983
439 0      10  2014
440 0       6  1989
441 0       9  1980
442 0       9  1992
443 0       9  2000
444 0       7  1981
445 0       1  2008
446 0       4  2002
447 0       7  1985
448 0       5  2010
449 0       8  2002
450 0       1  1994
451 0      12  2004
452 0       3  2003
453 0       7  1991
454 0       7  1982

[112 rows x 2 columns]

### Rows 455-500
yyyy

In [434]:
s.loc[455:].head()

455                                                  sHemmorage caused by probe in 1984 Medical History:\n
456    sHas been at MYH since his treaters in NE retired in 2000. Was seen in NE for 20 years. Previouy...
457    Pt joined Army reserves in 2001 and has 3 years left in this commitment.-Mental Status Exam Was ...
458    one sister from whom he is estranged due to her opiate dependence, legal conflict over mother's ...
459                     sSince 1998. Prior medication trials (including efficacy, reasons discontinued):\n
dtype: object

In [489]:
sixth = df['string'].str.extractall('[\s\b\w]?(\d{4})[\s\b\w]?')
sixth.loc[455:]

0
    match      
455 0      1984
456 0      2000
457 0      2001
458 0      1982
459 0      1998
460 0      2012
461 0      1991
462 0      1988
463 0      2014
464 0      2016
465 0      1976
466 0      1981
467 0      2011
468 0      1997
469 0      2003
470 0      1983
471 0      1999
472 0      2010
473 0      1975
474 0      1972
475 0      2015
476 0      1989
477 0      1994
478 0      1993
479 0      1996
480 0      2013
481 0      1974
482 0      1990
483 0      1995
484 0      2004
485 0      1987
486 0      1973
487 0      1992
488 0      1977
489 0      1985
490 0      2007
491 0      2009
492 0      1986
493 0      1978
494 0      2002
495 0      1979
496 0      2006
497 0      2008
498 0      2005
499 0      1980

### Together

In [802]:
dic = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}

# 0-124  (mm/dd/yyyy or mm/dd/yy)
first = df[:125]['string'].str.extractall(r'(?P<mm>\d{1,2})[/-](?P<dd>\d{1,2})[/-](?P<yyyy>\d{2,4}).*')
# Add 1900 to two digit years
for i, y in enumerate(first.yyyy):
    if len(y) == 2:
        first.yyyy[i] = str(int(y) + 1900)

# 125-193 (dd mon yyyy)
second = df[125:194]['string'].str.extractall('(?P<dd>\d{1,2} )?(?P<mm>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(?:[a-z]*) ?(?P<yyyy>\d{4})')
second.mm = second.mm.map(dic)

# 194-227 (mon dd yyyy)
third = df[194:228]['string'].str.extractall('(?P<mm>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(?:.*) (?P<dd>\d{1,2}).? (?P<yyyy>\d{4})')
third.mm = third.mm.map(dic)

# 228-342 (mon yyyy)
fourth = df[228:343]['string'].str.extractall('(?P<dd>\d{1,2} )?(?P<mm>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(?:[a-z]*)?(?:\D*)(?P<yyyy>\d{4})')
fourth.mm = fourth.mm.map(dic)
fourth['dd'] = 1

# 343-454 (mm/yyyy)
fifth = df[343:455]['string'].str.extractall('(?P<mm>\d{1,2})/(?P<yyyy>\d{4})')

# 455-500 (yyyy)
sixth = df[455:]['string'].str.extractall('[\s\b\w]?(\d{4})[\s\b\w]?')
sixth.columns = ['yyyy']  # Rename

regexs = [first, second, third, fourth, fifth, sixth]

for ex in regexs:
    # Get original index
    ex.reset_index(inplace=True)
    ex.drop('match', axis=1, inplace=True)
    ex.rename(columns={'level_0':'orig_index'}, inplace=True)

    # Fill missing columns and values
    if 'mm' not in ex.columns:
        ex['mm'] = 1
    if 'dd' not in ex.columns:
        ex['dd'] = 1    
    
# Vertical stack into one df
stack = pd.concat([first, second, third, fourth, fifth, sixth])

# Convert columns to one datetime
stack[['mm', 'dd', 'yyyy']] = stack[['mm', 'dd', 'yyyy']].astype('str')
stack['datetime'] = pd.to_datetime(stack.yyyy + ' ' + stack.mm + ' ' + stack.dd, 
                                   infer_datetime_format=True)

stack.sort_values('datetime', ascending=True)

# stack.sort_values('orig_index')

dd  mm  orig_index  yyyy   datetime
9     10   4           9  1971 1971-04-10
84    18   5          84  1971 1971-05-18
2      8   7           2  1971 1971-07-08
53    11   7          53  1971 1971-07-11
28    12   9          28  1971 1971-09-12
19     1   1         474  1972 1972-01-01
28   13    1         153  1972 1972-01-13
13    26   1          13  1972 1972-01-26
4    06    5         129  1972 1972-05-06
98    13   5          98  1972 1972-05-13
111   10   6         111  1972 1972-06-10
31    15   6         225  1972 1972-06-15
31    20   7          31  1972 1972-07-20
46   04   10         171  1972 1972-10-04
66   30   11         191  1972 1972-11-30
31     1   1         486  1973 1973-01-01
107    1   2         335  1973 1973-02-01
72     1   2         415  1973 1973-02-01
36    14   2          36  1973 1973-02-14
62     1  03         405  1973 1973-03-01
95     1   3         323  1973 1973-03-01
79     1   4         422  1973 1973-04-01
32     1  06         375  1973 1973-06-01
37     1   7         380  1973 1973-07-01
2      1  10         345  1973 1973-10-01
57    01  12          57  1973 1973-12-01
26     1   1         481  1974 1974-01-01
93     1   2         436  1974 1974-02-01
104   24   2         104  1974 1974-02-24
71     1   3         299  1974 1974-03-01
..   ...  ..         ...   ...        ...
26    25   6         220  2012 2012-06-25
14    01   9         208  2012 2012-09-01
15     1   9         243  2012 2012-09-01
14   21   10         139  2012 2012-10-21
92     1  11         320  2012 2012-11-01
40     1  12         383  2012 2012-12-01
16     1   1         244  2013 2013-01-01
58     1   1         286  2013 2013-01-01
25     1   1         480  2013 2013-01-01
88     1   4         431  2013 2013-04-01
51     1   9         279  2013 2013-09-01
4     11  10         198  2013 2013-10-11
38     1   1         381  2014 2014-01-01
8      1   1         463  2014 2014-01-01
23     1   7         366  2014 2014-07-01
96     1  10         439  2014 2014-10-01
27     1  10         255  2014 2014-10-01
58     1  12         401  2014 2014-12-01
20     1   1         475  2015 2015-01-01
29     1   9         257  2015 2015-09-01
27   28    9         152  2015 2015-09-28
7      1  10         235  2015 2015-10-01
9      1   1         464  2016 2016-01-01
25     1   2         253  2016 2016-02-01
84     1   5         427  2016 2016-05-01
3      1   5         231  2016 2016-05-01
16   30    5         141  2016 2016-05-30
61   13   10         186  2016 2016-10-13
36   19   10         161  2016 2016-10-19
70     1  11         413  2016 2016-11-01

[500 rows x 5 columns]

## Final Answer

In [5]:
def date_sorter():
    
    # Skeleton
    import pandas as pd
    import re
    pd.set_option('display.max_colwidth', 100)

    doc = []
    with open('dates.txt') as file:
        for line in file:
            doc.append(line)

    s = pd.Series(doc)    
    
    
    
    # Regular Expressions
    dic = {'Jan':1, 'Feb':2, 'Mar':3, 'Apr':4, 'May':5, 'Jun':6, 'Jul':7, 'Aug':8, 'Sep':9, 'Oct':10, 'Nov':11, 'Dec':12}

    # 0-124  (mm/dd/yyyy or mm/dd/yy)
    first = df[:125]['string'].str.extractall(r'(?P<mm>\d{1,2})[/-](?P<dd>\d{1,2})[/-](?P<yyyy>\d{2,4}).*')
    # Add 1900 to two digit years
    for i, y in enumerate(first.yyyy):
        if len(y) == 2:
            first.yyyy[i] = str(int(y) + 1900)

    # 125-193 (dd mon yyyy)
    second = df[125:194]['string'].str.extractall('(?P<dd>\d{1,2} )?(?P<mm>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(?:[a-z]*) ?(?P<yyyy>\d{4})')
    second.mm = second.mm.map(dic)

    # 194-227 (mon dd yyyy)
    third = df[194:228]['string'].str.extractall('(?P<mm>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(?:.*) (?P<dd>\d{1,2}).? (?P<yyyy>\d{4})')
    third.mm = third.mm.map(dic)

    # 228-342 (mon yyyy)
    fourth = df[228:343]['string'].str.extractall('(?P<dd>\d{1,2} )?(?P<mm>Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(?:[a-z]*)?(?:\D*)(?P<yyyy>\d{4})')
    fourth.mm = fourth.mm.map(dic)
    fourth['dd'] = 1

    # 343-454 (mm/yyyy)
    fifth = df[343:455]['string'].str.extractall('(?P<mm>\d{1,2})/(?P<yyyy>\d{4})')

    # 455-500 (yyyy)
    sixth = df[455:]['string'].str.extractall('[\s\b\w]?(\d{4})[\s\b\w]?')
    sixth.columns = ['yyyy']  # Rename

    # Cleaning
    regexs = [first, second, third, fourth, fifth, sixth]

    for ex in regexs:
        # Get original index
        ex.reset_index(inplace=True)
        ex.drop('match', axis=1, inplace=True)
        ex.rename(columns={'level_0':'orig_index'}, inplace=True)

        # Fill missing columns and values
        if 'mm' not in ex.columns:
            ex['mm'] = 1
        if 'dd' not in ex.columns:
            ex['dd'] = 1    

    # Vertical stack into one df
    stack = pd.concat([first, second, third, fourth, fifth, sixth])

    # Convert columns to one datetime
    stack[['mm', 'dd', 'yyyy']] = stack[['mm', 'dd', 'yyyy']].astype('str')
    stack['datetime'] = pd.to_datetime(stack.yyyy + ' ' + stack.mm + ' ' + stack.dd, 
                                       infer_datetime_format=True)

    stack = stack.sort_values('datetime', ascending=True)
    series = pd.Series(stack['orig_index']).reset_index(drop=True)
    
    return series

In [6]:
# date_sorter()

## Sophie's Test

In [7]:
def test():
    import pandas as pd

    fun = date_sorter()
    res = 'Data Type Test (Series?): '
    res += ['Failed\n','Passed\n'][type(fun)==pd.Series]
    
    res += 'Data Shape Test ((500,)?): '
    res += ['Failed\n','Passed\n'][fun.shape==(500,)]
    
    res += 'Index Values Test (range(500)?): '
    res += ['Failed\n','Passed\n'][fun.index.tolist()==list(range(500))]
    
    res += 'Values Test (0-499): '
    res += ['Failed\n','Passed\n'][all((fun<500) & (fun>=0))]
    
    return res

# print(test())   